## Convolutional neural network (CNN)<a name="convolutional"></a>
[[go back to the top]](#contents)

For the CNN classifier, we have two options:
- **1D CNN**, which inputs can be obtained by applying the **CNN directly on portions (windows)** of the original sound signal (after downsampling and normalization).
- **2D CNN**, which are based on **time frequency analysis of sounds**, as the **Mel-frequency cepstral coefficients (MFCCs)**.

With the features extracted above, we can evaluate their suitability for each CNN approach.
- **1D CNN:**
    - Spectral Centroid
    - Spectral Bandwith
    - Spectral Flatness
    - Spectral Rollof
- **2D CNN:**
    - Chromagram
    - Mel-Scaled Spectogram
    - Short-time Fourier transform Tempogram